In [1]:
from loader import MoleculeDataset, SeqDataset,SeqMolDataset#########################
import torch
import torch
from torchvision.models import resnet18

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import argparse

from loader import MoleculeDataset#################
#from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
import numpy as np

from model import GNN, GNN_graphpred,GNN_graphpred_1
from sklearn.metrics import roc_auc_score

from splitters import scaffold_split,scaffold_split_1
import pandas as pd
import os
import shutil
from tensorboardX import SummaryWriter
import esm

from SeqMolModel import InteractionModel,InteractionModel_1

In [2]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch implementation of pre-training of graph neural networks')
parser.add_argument('--device', type=int, default=0,
                        help='which gpu to use if any (default: 0)')#0000
parser.add_argument('--batch_size', type=int, default=16,
                        help='input batch size for training (default: 32)')
parser.add_argument('--epochs', type=int, default=100,
                        help='number of epochs to train (default: 100)')
parser.add_argument('--lr', type=float, default=0.001,
                        help='learning rate (default: 0.001)')
parser.add_argument('--lr_scale', type=float, default=1,
                        help='relative learning rate for the feature extraction layer (default: 1)')
parser.add_argument('--decay', type=float, default=0,
                        help='weight decay (default: 0)')
parser.add_argument('--num_layer', type=int, default=5,
                        help='number of GNN message passing layers (default: 5).')
parser.add_argument('--emb_dim', type=int, default=300,
                        help='embedding dimensions (default: 300)')
parser.add_argument('--dropout_ratio', type=float, default=0.5,
                        help='dropout ratio (default: 0.5)')
parser.add_argument('--graph_pooling', type=str, default="mean",
                        help='graph level pooling (sum, mean, max, set2set, attention)')
parser.add_argument('--JK', type=str, default="last",
                        help='how the node features across layers are combined. last, sum, max or concat')
parser.add_argument('--gnn_type', type=str, default="gin")
parser.add_argument('--dataset', type=str, default = 'sider', help='root directory of dataset. For now, only classification.')
#parser.add_argument('--input_model_file', type=str, default = 'None', help='filename to read the model (if there is any)')
parser.add_argument('--input_model_file', type=str, default = 'Mole-BERT', help='filename to read the model (if there is any)')
parser.add_argument('--filename', type=str, default = '', help='output filename')
parser.add_argument('--seed', type=int, default=42, help = "Seed for splitting the dataset.")
parser.add_argument('--runseed', type=int, default=0, help = "Seed for minibatch selection, random initialization.")
parser.add_argument('--split', type = str, default="scaffold", help = "random or scaffold or random_scaffold")
parser.add_argument('--eval_train', type=int, default = 1, help='evaluating training or not')
parser.add_argument('--num_workers', type=int, default = 4, help='number of workers for dataset loading')
args = parser.parse_args(args=[])###############33


In [3]:
device = torch.device("cuda:" + str(args.device)) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(args.runseed)

num_tasks=1
# Load ESM-2 model
protein_model, protein_alphabet = esm.pretrained.esm2_t33_650M_UR50D()

print(protein_alphabet)
#alphabet = esm.Alphabet.from_architecture(model_data["args"].arch)
#batch_converter = alphabet.get_batch_converter()
protein_model.eval()  # disables dropout for deterministic results

#self.molecular.model,self.molecular.node_representation,self.molecular.features = GNN_graphpred_1(args.num_layer, args.emb_dim, num_tasks, JK = args.JK, drop_ratio = args.dropout_ratio, graph_pooling = args.graph_pooling, gnn_type = args.gnn_type)
molecular_model = GNN_graphpred(args.num_layer, args.emb_dim, num_tasks, JK = args.JK, drop_ratio = args.dropout_ratio, graph_pooling = args.graph_pooling, gnn_type = args.gnn_type)
###################################
if not args.input_model_file == "None":###############
    print('Not from scratch')
    molecular_model.from_pretrained('model_gin/{}.pth'.format(args.input_model_file))
    print('rese:model_gin')
molecular_model.to(device)



model_param_group = []
model_param_group.append({"params": molecular_model.gnn.parameters()})
if args.graph_pooling == "attention":
    model_param_group.append({"params": molecular_model.pool.parameters(), "lr":args.lr*args.lr_scale})
'''
————————————————
版权声明：本文为CSDN博主「Dreamcatcher风」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/Wind_2028/article/details/120541017   
'''



model_param_group.append({"params": molecular_model.graph_pred_linear.parameters(), "lr":args.lr*args.lr_scale})
optimizer = optim.Adam(model_param_group, lr=0.01, weight_decay=args.decay)#############

Not from scratch
rese:model_gin


In [4]:
#num_tasks=1
model=InteractionModel_1(protein_model=protein_model,molecular_model=molecular_model,mol_emb_dim=128,pro_emb_dim=1028,num_tasks=1) 
model.to(device)
print(model)#nice

InteractionModel_1(
  (mole_model): GNN(
    (x_embedding1): Embedding(120, 300)
    (x_embedding2): Embedding(3, 300)
    (gnns): ModuleList(
      (0): GINConv()
      (1): GINConv()
      (2): GINConv()
      (3): GINConv()
      (4): GINConv()
    )
    (batch_norms): ModuleList(
      (0): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (pred_linear): Linear(in_features=1156, out_features=1, bias=True)
)


In [5]:
gnn_dataset = MoleculeDataset("./dataset/" + args.dataset, dataset=args.dataset)###########################转换成了分子图的格式

In [6]:
seq_dataset=SeqDataset('dataset/affinity/processed/sequence.csv')
#seq_dataset[2]

In [7]:
#########error
'''
from torch.utils.data import DataLoader 
from torch_geometric.data import DataLoader as GeometricDataLoader




class MultiLoader:
    def __init__(self, dataloader1, dataloader2):
        self.dataloader1 = dataloader1
        self.dataloader2 = dataloader2

    def __iter__(self):
        # Ensure both shuffles are synchronized
        self.dataloader1.shuffle = self.dataloader2.shuffle

        for data1, data2 in zip(self.dataloader1, self.dataloader2):
            yield data1, data2

    def __len__(self):
        return min(len(self.dataloader1), len(self.dataloader2))

# Example usage
if __name__ == "__main__":
    # Create DataLoader instances (replace with your actual datasets and configurations)
    dataloader1 = DataLoader(seq_dataset, batch_size=32, shuffle=True)
    dataloader2 = GeometricDataLoader(gnn_dataset, batch_size=32, shuffle=True)

    multi_loader = MultiLoader(dataloader1, dataloader2)

    for batch_data1, batch_data2 in multi_loader:
        # Process your batch data here
        print("Data from DataLoader 1:", batch_data1)
        print("Data from DataLoader 2:", batch_data2)
'''

'\nfrom torch.utils.data import DataLoader \nfrom torch_geometric.data import DataLoader as GeometricDataLoader\n\n\n\n\nclass MultiLoader:\n    def __init__(self, dataloader1, dataloader2):\n        self.dataloader1 = dataloader1\n        self.dataloader2 = dataloader2\n\n    def __iter__(self):\n        # Ensure both shuffles are synchronized\n        self.dataloader1.shuffle = self.dataloader2.shuffle\n\n        for data1, data2 in zip(self.dataloader1, self.dataloader2):\n            yield data1, data2\n\n    def __len__(self):\n        return min(len(self.dataloader1), len(self.dataloader2))\n\n# Example usage\nif __name__ == "__main__":\n    # Create DataLoader instances (replace with your actual datasets and configurations)\n    dataloader1 = DataLoader(seq_dataset, batch_size=32, shuffle=True)\n    dataloader2 = GeometricDataLoader(gnn_dataset, batch_size=32, shuffle=True)\n\n    multi_loader = MultiLoader(dataloader1, dataloader2)\n\n    for batch_data1, batch_data2 in multi_l

In [10]:
#chartGPT太厉害了，杀了我吧
from torch.utils.data import DataLoader
from torch_geometric.data import DataLoader as GeometricDataLoader

class MultiDataLoader:
    def __init__(self, dataloader1, dataloader2):
        self.dataloader1 = dataloader1
        self.dataloader2 = dataloader2

    def __iter__(self):
        for data1, data2 in zip(self.dataloader1, self.dataloader2):
            yield data1, data2

    def __len__(self):
        return min(len(self.dataloader1), len(self.dataloader2))

    def set_shuffle(self, shuffle):
        self.dataloader1.shuffle = shuffle
        self.dataloader2.shuffle = shuffle

# Example usage
if __name__ == "__main__":
    # Create DataLoader instances (replace with your actual datasets and configurations)
    dataloader1 = DataLoader(seq_dataset, batch_size=32, shuffle=True)
    dataloader2 = GeometricDataLoader(gnn_dataset, batch_size=32, shuffle=True)

    multi_loader = MultiDataLoader(dataloader1, dataloader2)

    # Set the shuffle parameter simultaneously for both dataloaders
    multi_loader.set_shuffle(True)

    for batch_data1, batch_data2 in multi_loader:
        # Process your batch data here
        print("Data from DataLoader 1:", batch_data1)
        print("Data from DataLoader 2:", batch_data2)


Data from DataLoader 1: ['SELLVNTKSGKVMGTRVPVLSSHISAFLGIPFAEPPVGNMRFRRPEPKKPWSGVWNASTYPNNCQQYVDEQFPGFSGSEMWNPNREMSEDCLYLNIWVPSPRPKSTTVMVWIYGGGFYSGSSTLDVYNGKYLAYTEEVVLVSLSYRVGAFGFLALHGSQEAPGNVGLLDQRMALQWVHDNIQFFGGDPKTVTIFGESAGGASVGMHILSPGSRDLFRRAILQSGSPNCPWASVSVAEGRRRAVELGRNLNCNLNSDEELIHCLREKKPQELIDVEWNVLPFDSIFRFSFVPVIDGEFFPTSLESMLNSGNFKKTQILLGVNKDEGSFFLLYGAPGFSKDSESKISREDFMSGVKLSVPHANDLGLDAVTLQYTDWMDDNNGIKNRDGLDDIVGDHNVICPLMHFVNKYTKFGNGTYLYFFNHRASNLVWPEWMGVIHGYEIEFVFGLPLVKELNYTAEEEALSRRIMHYWATFAKTGNPNEPHS', 'VKKFPEGFLWGVATASYQIEGSPLADGAGMSIWHTFSHTPGNVKNGDTGDVACDHYNRWKEDIEIIEKLGVKAYRFSISWPRILPEGTGRVNQKGLDFYNRIIDTLLEKGITPFVTIYHWDLPFALQLKGGWANREIADWFAEYSRVLFENFGDRVKNWITLNEPWVVAIVGHLYGVHAPGMRDIYVAFRAVHNLLRAHARAVKVFRETVKDGKIGIVFNNGYFEPASEK', 'ASETFEFQAEITQLMSLIINTVYSNKEIFLRELISNASDALDKIRYKSLSDPKQLETEPDLFIRITPKPEQKVLEIRDSGIGMTKAELINNLGTIAKSGTKAFMEALSAGADVSMIGQFGVGFYSLFLVADRVQVISKSNDDEQYIWESNAGGSFTVTLDEVNERIGRGTILRLFLKDDQLEYLEEKRIKEVIKRHSEFVAYPIQLVVTKEVE', 'DQPMEEEEVETFAFQAEIAQLMSLIINTFYSNKEI

/home/zhengfang/anaconda3/envs/new_pytorch/lib/python3.6/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [11]:
#####test with chartGPT  DataSet extends two parents' classes

import torch
from torch.utils.data import Dataset
from torch_geometric.data import InMemoryDataset

class MultiDatasetMixin:
    def __init__(self, dataset1, dataset2):
        self.dataset1 = dataset1
        self.dataset2 = dataset2

    def __len__(self):
        return min(len(self.dataset1), len(self.dataset2))

    def __getitem__(self, idx):
        
        return self.dataset1[idx], self.dataset2[idx]

class CustomMultiDataset(MultiDatasetMixin, Dataset):
    def __init__(self, dataset1, dataset2):
        MultiDatasetMixin.__init__(self, dataset1, dataset2)




# Example usage
if __name__ == "__main__":

    
    class DummyDataset1(Dataset):
        def __len__(self):
            return 100

        def __getitem__(self, idx):
            return torch.rand(3)

    class DummyDataset2(InMemoryDataset):
        def __init__(self):
            super(DummyDataset2, self).__init__("path/to/processed/dataset")
            self.data = None  # Replace with actual data loading logic

    dataset1 = DummyDataset1()
    dataset2 = DummyDataset2()
    
    
    dataset1=seq_dataset
    dataset2=gnn_dataset
    custom_dataset = CustomMultiDataset(dataset1, dataset2)

    print(len(custom_dataset))
    print(custom_dataset[0])


206
('HWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFCFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFNGEGEPEELMVDNWRPAQPLKNRQIKASFK', Data(id=[1], edge_index=[2, 24], x=[13, 2], y=[27], edge_attr=[24, 2]))


In [10]:
'''
import torch
from torch.utils.data import DataLoader 
from torch_geometric.data import DataLoader as GeometricDataLoader######as

class MultiDataLoader(DataLoader,GeometricDataLoader):
    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):
        # Initialize the parent DataLoader class
        DataLoader.__init__(self, CustomMultiDataset(dataset1, dataset2), batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        #DataLoader.__init__(self, MultiDatasetMixin(dataset1, dataset2), batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
# Example usage
if __name__ == "__main__":
    
    class DummyDataset1(Dataset):
        def __len__(self):
            return 100

        def __getitem__(self, idx):
            return torch.rand(3)

    class DummyDataset2(InMemoryDataset):
        def __init__(self):
            super(DummyDataset2, self).__init__("path/to/processed/dataset")
            self.data = None  # Replace with actual data loading logic

    dataset1 = DummyDataset1()
    dataset2 = DummyDataset2()
    
    dataset1=seq_dataset
    dataset2=gnn_dataset
    
    batch_size = 32
    shuffle = True
    num_workers = 4

    dataloader = MultiDataLoader(dataset1, dataset2, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    for batch in dataloader:
        # Process your batch data here
        data1, data2 = batch
        print("Data from Dataset 1:", data1)
        print("Data from Dataset 2:", data2)
'''

'\nimport torch\nfrom torch.utils.data import DataLoader \nfrom torch_geometric.data import DataLoader as GeometricDataLoader######as\n\nclass MultiDataLoader(DataLoader,GeometricDataLoader):\n    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):\n        # Initialize the parent DataLoader class\n        DataLoader.__init__(self, CustomMultiDataset(dataset1, dataset2), batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)\n        #DataLoader.__init__(self, MultiDatasetMixin(dataset1, dataset2), batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)\n# Example usage\nif __name__ == "__main__":\n    \n    class DummyDataset1(Dataset):\n        def __len__(self):\n            return 100\n\n        def __getitem__(self, idx):\n            return torch.rand(3)\n\n    class DummyDataset2(InMemoryDataset):\n        def __init__(self):\n            super(DummyDataset2, self).__init__("path/to/processed/dataset")\n            self.data = N

In [11]:
'''
import torch_geometric
seq_loader=torch.utils.data.DataLoader(dataset1,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=True)
gnn_loader=torch_geometric.loader.DataLoader(dataset2,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=True)

for seq in seq_loader:
    print(seq)
    
for gnn in gnn_loader:
    print(gnn)
'''

'\nimport torch_geometric\nseq_loader=torch.utils.data.DataLoader(dataset1,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=True)\ngnn_loader=torch_geometric.loader.DataLoader(dataset2,batch_size=args.batch_size,num_workers=args.num_workers,shuffle=True)\n\nfor seq in seq_loader:\n    print(seq)\n    \nfor gnn in gnn_loader:\n    print(gnn)\n'

import torch
from torch.utils.data import DataLoader
from torch_geometric.data import DataLoader as GeometricDataLoader



class MultiDataLoader(DataLoader):
    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):
        # Initialize the parent DataLoader class
        DataLoader.__init__(self, CustomMultiDataset(dataset1, dataset2), batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

# Example usage

if __name__ == "__main__":
    
    class DummyDataset1(Dataset):
        def __len__(self):
            return 100

        def __getitem__(self, idx):
            return torch.rand(3)

    class DummyDataset2(InMemoryDataset):
        def __init__(self):
            super(DummyDataset2, self).__init__("path/to/processed/dataset")
            self.data = None  # Replace with actual data loading logic

    dataset1 = DummyDataset1()
    dataset2 = DummyDataset2()
    
    dataset1=seq_dataset
    dataset2=gnn_dataset
    #custom_dataset = CustomMultiDataset(dataset1, dataset2)
    batch_size = 32
    shuffle = True
    num_workers = 4

    dataloader = MultiDataLoader(dataset1, dataset2, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

    for batch in dataloader:
        # Process your batch data here
        data1, data2 = batch
        print("Data from Dataset 1:", data1)
        print("Data from Dataset 2:", data2)


In [12]:
'''
import torch
from torch.utils.data import Dataset
from torch_geometric.data import InMemoryDataset
import torch
from torch.utils.data import DataLoader as torchDataLoader
from torch_geometric.data import DataLoader as GeometricDataLoader#######
class MultiDatasetMixin:
    def __init__(self, dataset1, dataset2):
        self.dataset1 = dataset1
        self.dataset2 = dataset2

    def __len__(self):
        return min(len(self.dataset1), len(self.dataset2))

    def __getitem__(self, idx):
        return self.dataset1[idx], self.dataset2[idx]

class CustomMultiDataset(MultiDatasetMixin, Dataset):
    def __init__(self, dataset1, dataset2):
        MultiDatasetMixin.__init__(self, dataset1, dataset2)
        
        


class MultiDataLoader(DataLoader):
    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):
        # Initialize the parent DataLoader class
        #DataLoader.__init__(self, CustomMultiDataset(dataset1, dataset2), batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        
        DataLoader.__init__(self, MultiDatasetMixin(dataset1, dataset2), batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
        
        
    def __next__(self):
        index = next(self.batch_sampler)
        data = [self.dataset[idx] for idx in index]
        data = self.collate_fn(data)
        return data

    # 返回自身，因为自身实现了 __next__ 
    def __iter__(self):
        return self



class MultiDataLoader(DataLoader):
    def __init__(self, dataset, batch_size=1, shuffle=False, sampler=None,
                 batch_sampler=None, collate_fn=None, drop_last=False):
        self.dataset = dataset
        if batch_sampler is None:
            batch_sampler = BatchSampler(sampler, batch_size, drop_last)
        self.batch_size = batch_size
        self.drop_last = drop_last
        self.sampler = sampler
        self.batch_sampler = iter(batch_sampler)

        if collate_fn is None:
            collate_fn = default_collate
        self.collate_fn = collate_fn
        
    # 核心代码
    def __next__(self):
        index = next(self.batch_sampler)
        data1,data2 = [self.dataset[idx] for idx in index]
        data = self.collate_fn(data)
        return data1,data2
     # 返回自身，因为自身实现了 __next__ 
    def __iter__(self):
        return self
    
class MultiDataLoader:
    
    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):
        self.dataset1=dataset1
        self.dataset2=dataset2
        self.dataloader1=torchDataLoader(dataset1, batch_size,shuffle,num_workers)
        self.dataloader2=GeometricDataLoader(dataset2,batch_size,shuffle,num_workers)
        self.batch_size=batch_size
        self.shuffle=shuffle
        self.num_workers=num_workers
    def __getitem__(self,idx):
        return self.dataloader1[idx].self.dataloader2
    
    def __len__(self):
        return len(self.dataloader1)
       
    def __next__(self):
        
        return data1, data2
    def __iter__(self):
        return self
    
class CustomMultiDataLoader(MultiDataLoader, DataLoader):
    def __init__(self, dataset1, dataset2,batch_size,shuffle,num_workers):
        MultiDataLoader.__init__(self, dataset1, dataset2,batch_size,shuffle,num_workers)

        
        
dataloader1,dataloader2=CustomMultiDataLoader(seq_dataset,gnn_dataset,batch_size=args.batch_size,shuffle=False,num_workers=args.num_workers)
for seq,gnn in zip(dataloader1,dataloader2):
    print(seq)
    print(gnn)
'''

'\nimport torch\nfrom torch.utils.data import Dataset\nfrom torch_geometric.data import InMemoryDataset\nimport torch\nfrom torch.utils.data import DataLoader as torchDataLoader\nfrom torch_geometric.data import DataLoader as GeometricDataLoader#######\nclass MultiDatasetMixin:\n    def __init__(self, dataset1, dataset2):\n        self.dataset1 = dataset1\n        self.dataset2 = dataset2\n\n    def __len__(self):\n        return min(len(self.dataset1), len(self.dataset2))\n\n    def __getitem__(self, idx):\n        return self.dataset1[idx], self.dataset2[idx]\n\nclass CustomMultiDataset(MultiDatasetMixin, Dataset):\n    def __init__(self, dataset1, dataset2):\n        MultiDatasetMixin.__init__(self, dataset1, dataset2)\n        \n        \n\n\nclass MultiDataLoader(DataLoader):\n    def __init__(self, dataset1, dataset2, batch_size=1, shuffle=False, num_workers=0):\n        # Initialize the parent DataLoader class\n        #DataLoader.__init__(self, CustomMultiDataset(dataset1, data

In [12]:

seq_gnn_dataset=MultiDatasetMixin(seq_dataset,gnn_dataset)

if args.split == "scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        
        #print('smiles_list:',smiles_list)
        mol_train_dataset, mol_valid_dataset, mol_test_dataset ,seq_train_dataset,seq_valid_dataset,seq_test_dataset= scaffold_split_1(seq_gnn_dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1)##########dataset
        print("scaffold")
elif args.split == "random":
        train_dataset, valid_dataset, test_dataset = random_split(dataset, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random")
elif args.split == "random_scaffold":
        smiles_list = pd.read_csv('./dataset/' + args.dataset + '/processed/smiles.csv', header=None)[0].tolist()
        train_dataset, valid_dataset, test_dataset = random_scaffold_split(dataset, smiles_list, null_value=0, frac_train=0.8,frac_valid=0.1, frac_test=0.1, seed = args.seed)
        print("random scaffold")
else:
        raise ValueError("Invalid split option.")

print('++++++++++', mol_train_dataset[0])


#seq_mol_train_dataset=MultiDatasetMixini(seq_train_dataset,mol_train_dataset)
#seq_mol_valid_dataset=SeqMolDataset(seq_valid_dataset,mol_valid_dataset)
#seq_mol_test_dataset=SeqMolDataset(seq_train_dataset,mol_test_dataset)


seq_mol_train_loader = MultiDataLoader(seq_train_dataset,mol_train_dataset,batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
seq_mol_val_loader = MultiDataLoader(seq_valid_dataset,mol_valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)
seq_mol_test_loader = MultiDataLoader(seq_test_dataset,mol_test_dataset,batch_size=args.batch_size, shuffle=False, num_workers = args.num_workers)

seq_train_dataset: <class 'pandas.core.series.Series'>
scaffold
++++++++++ Data(id=[1], edge_index=[2, 24], x=[13, 2], y=[27], edge_attr=[24, 2])


TypeError: __init__() got an unexpected keyword argument 'batch_size'

In [14]:
def train(args, epoch, model, device, loader, optimizer):
    model.train()
    #epoch_iter = tqdm(loader, desc="Iteration")
    for step, (A,B) in enumerate(loader):
        print('A:',A)
        print('B:',B)
        A = A.to(device)
        B=B.to(device)
        #print('batch:',batch.x,batch.edge_index,batch.edge_attr,batch.batch)
        pred=model(A,*B)
        #pred, node_representation = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)
        y = B.y.view(pred.shape).to(torch.float64)

        #Whether y is non-null or not.
        is_valid = y**2 > 0
        #Loss matrix
        loss_mat = criterion(pred.double(), (y+1)/2)
        #loss matrix after removing null target
        loss_mat = torch.where(is_valid, loss_mat, torch.zeros(loss_mat.shape).to(loss_mat.device).to(loss_mat.dtype))  
        optimizer.zero_grad()
        loss = torch.sum(loss_mat)/torch.sum(is_valid)
        loss.backward()
        optimizer.step()
        epoch_iter.set_description(f"Epoch: {epoch} tloss: {loss:.4f}")


In [15]:
train_acc_list = []
val_acc_list = []
test_acc_list = []

if not args.filename == "":
    fname = 'runs/seq_mol_finetune_cls_runseed' + str(args.runseed) + '/' + args.filename
    #delete the directory if there exists one
    if os.path.exists(fname):
        shutil.rmtree(fname)
        print("removed the existing file.")
    writer = SummaryWriter(fname)

for epoch in range(1, args.epochs+1):
    print("====epoch " + str(epoch))
        
    train(args, epoch, model, device, seq_mol_train_loader, optimizer)

    print("====Evaluation")
    if args.eval_train:
        train_acc = eval(args, model, device, train_loader)
    else:
        print("omit the training accuracy computation")
        train_acc = 0
    val_acc = eval(args, model, device, val_loader)
    test_acc = eval(args, model, device, test_loader)

    print("train: %f val: %f test: %f" %(train_acc, val_acc, test_acc))
    val_acc_list.append(val_acc)
    test_acc_list.append(test_acc)
    train_acc_list.append(train_acc)

    if not args.filename == "":
        writer.add_scalar('data/train auc', train_acc, epoch)
        writer.add_scalar('data/val auc', val_acc, epoch)
        writer.add_scalar('data/test auc', test_acc, epoch)

print('Best epoch:', val_acc_list.index(max(val_acc_list)))
print('Best auc: ', test_acc_list[val_acc_list.index(max(val_acc_list))])

exp_path = os.getcwd() + '/{}_results/{}/'.format(args.input_model_file, args.dataset)
if not os.path.exists(exp_path):
    os.makedirs(exp_path)

df = pd.DataFrame({'train':train_acc_list,'valid':val_acc_list,'test':test_acc_list})
df.to_csv(exp_path + 'seed{}.csv'.format(args.runseed))

logs = 'Dataset:{}, Seed:{}, Best Epoch:{}, Best Acc:{:.5f}'.format(args.dataset, args.runseed, val_acc_list.index(max(val_acc_list)), test_acc_list[val_acc_list.index(max(val_acc_list))])
with open(exp_path + '{}_log.csv'.format(args.dataset),'a+') as f:
    f.write('\n')
    f.write(logs)

if not args.filename == "":
    writer.close()

====epoch 1
key:key: id 
iditem,slices:
key: item,slices:  idtensor([   0,    1,    2,  ..., 1424, 1425, 1426])
tensor([   0,    1,    2,  ..., 1424, 1425, 1426])item,slices:  tensor([   0,    1,    2,  ..., 1425, 1426, 1427])key:tensor([   0,    1,    2,  ..., 1425, 1426, 1427]) 

 key:idtensor([   0,    1,    2,  ..., 1424, 1425, 1426])key: 
  ytensor([   0,    1,    2,  ..., 1425, 1426, 1427])yitem,slices: 

key:
item,slices:tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) item,slices:   tensor([ 1,  1, -1,  ...,  1,  1,  1])ytensor([ 1,  1, -1,  ...,  1,  1,  1])
tensor([   0,    1,    2,  ..., 1425, 1426, 1427])  
tensor([    0,    27,    54,  ..., 38475, 38502, 38529])key:item,slices:tensor([    0,    27,    54,  ..., 38475, 38502, 38529])

  key:key:y tensor([ 1,  1, -1,  ...,  1,  1,  1]) edge_attr
edge_attr item,slices:

 tensor([    0,    27,    54,  ..., 38475, 38502, 38529])item,slices:
item,slices:tensor([ 1,  1, -1,  ...,  1,  1,  1])key:   tensor([[0, 0],
        [0, 0

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/zhengfang/anaconda3/envs/new_pytorch/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2891, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1032, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1039, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/zhengfang/anaconda3/envs/new_pytorch/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/zhengfang/anaconda3/envs/ne

 
tensor([   0,    1,    2,  ..., 1425, 1426, 1427])tensor([   0,    1,    2,  ..., 1424, 1425, 1426])
key:key: 
 idy

item,slices:item,slices: tensor([ 1,  1, -1,  ...,  1,  1,  1])  tensor([    0,    27,    54,  ..., 38475, 38502, 38529]) tensor([   0,    1,    2,  ..., 1424, 1425, 1426])key: 
tensor([   0,    1,    2,  ..., 1425, 1426, 1427])tensor([   0,    1,    2,  ..., 1425, 1426, 1427])key:

 key: key: edge_attrid 
yyitem,slices:

item,slices:
item,slices: item,slices:   tensor([ 1,  1, -1,  ...,  1,  1,  1])tensor([ 1,  1, -1,  ...,  1,  1,  1])tensor([   0,    1,    2,  ..., 1424, 1425, 1426])   tensor([   0,    1,    2,  ..., 1425, 1426, 1427])tensor([    0,    27,    54,  ..., 38475, 38502, 38529])tensor([    0,    27,    54,  ..., 38475, 38502, 38529])tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]])


key:key:key:    yedge_attredge_attrtensor([     0,     24,     86,  ..., 100788, 100864, 100912])


item,slices:i

        [3, 0]])tensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]])key:    tensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]])tensor([     0,     24,     86,  ..., 100788, 100864, 100912])tensor([     0,     24,     86,  ..., 100788, 100864, 100912])edge_attr

 
key:key:item,slices:   idtensor([     0,     24,     86,  ..., 100788, 100864, 100912])tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]])

x 
item,slices:item,slices:key:tensor([     0,     24,     86,  ..., 100788, 100864, 100912]) tensor([   0,    1,    2,  ..., 1424, 1425, 1426]) 
  key: tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]])idtensor([   0,    1,    2,  ..., 1425, 1426, 1427]) 

xitem,slices:
tensor([    0,    13,    42,  ..., 47949, 47983, 48006])key: 
 item,slices:ykey:tensor([   0,    1,    2,  ..., 1424, 1425, 1426])   
item,slices:tens

  key:key:tensor([     0,     24,     86,  ..., 100788, 100864, 100912])  
xidkey:
tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
item,slices:
 item,slices:key:id  tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]]) 
tensor([   0,    1,    2,  ..., 1424, 1425, 1426])  edge_attritem,slices:
tensor([    0,    13,    42,  ..., 47949, 47983, 48006])tensor([   0,    1,    2,  ..., 1425, 1426, 1427]) item,slices:

key: tensor([   0,    1,    2,  ..., 1424, 1425, 1426])  key:ytensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]])
tensor([   0,    1,    2,  ..., 1425, 1426, 1427])  edge_indexitem,slices:
tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
key:
  item,slices:key:  tensor([ 1,  1, -1,  ...,  1,  1,  1]) ytensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]])x
tensor([    0,    27,    54,  ..., 38475, 38502, 38529])
 
item

        [3, 0]])edge_index

 key:item,slices:item,slices:  tensor([     0,     24,     86,  ..., 100788, 100864, 100912]) 
tensor([ 1,  1, -1,  ...,  1,  1,  1])xtensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]])
 key: item,slices:tensor([    0,    27,    54,  ..., 38475, 38502, 38529])  tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]])
x key:key:
tensor([    0,    13,    42,  ..., 47949, 47983, 48006]) item,slices:
edge_attrkey: 
 item,slices:edge_index id
 
item,slices:tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]]) tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]])item,slices:   tensor([[ 0,  1,  1,  ..., 11, 22, 17],
        [ 1,  0,  2,  ..., 16, 17, 22]])tensor([     0,     24,     86,  ..., 100788, 100864, 100

item,slices:item,slices:  tensor([ 1,  1, -1,  ...,  1,  1,  1])tensor([   0,    1,    2,  ..., 1424, 1425, 1426])  tensor([    0,    27,    54,  ..., 38475, 38502, 38529])tensor([   0,    1,    2,  ..., 1425, 1426, 1427])

key:key:  edge_attry

item,slices:item,slices:  tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]])tensor([ 1,  1, -1,  ...,  1,  1,  1])  tensor([     0,     24,     86,  ..., 100788, 100864, 100912])tensor([    0,    27,    54,  ..., 38475, 38502, 38529])

key:key:  x
edge_attritem,slices: 
item,slices:tensor([[5, 0],
        [5, 0],
        [6, 0],
        ...,
        [5, 0],
        [5, 0],
        [5, 0]])  tensor([    0,    13,    42,  ..., 47949, 47983, 48006])tensor([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [3, 0],
        [3, 0],
        [3, 0]])
 key: tensor([     0,     24,     86,  ..., 100788, 100864, 100912])
edge_indexkey:
 item,slices:x 
item,slices:tensor([[ 0,  1,  1,  .